<a href="https://colab.research.google.com/github/raima2000/Pharmacovigilance_detection/blob/main/Roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


In [ ]:
df=pd.read_excel("/content/psyTAR_selected.xlsx")

In [ ]:
df

,comment,ADR_Status
0,I am detoxing from Lexapro now. I slowly cut m...,ADR
1,Just TWO tablets of Lexapro 10mg completely de...,ADR
2,Be careful with this medication. This was my ...,ADR
3,"It didn't help me out, at all. My anxiety is w...",ADR
4,I would not suggest taking this medication. I ...,ADR
...,...,...
763,I almost lost everyone in my life that I loved...,ADR
764,"as good as it gets, at a price",Non-ADR
765,No longer have crying jags. Able to deal with...,ADR
766,I have been taking 75mg dosage for a year now ...,ADR


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['ADR_Status'] = label_encoder.fit_transform(df['ADR_Status'])

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.1 MB/s eta 0:00:00


In [ ]:
!pip install transformers tensorflow

In [ ]:
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification, RobertaConfig
import tensorflow as tf
import pandas as pd
import numpy as np

# Load your dataset (assuming you have a DataFrame named 'df')
# Make sure it contains 'Posts' and 'type_encoded' columns
# Example:
# df = pd.read_csv('your_dataset.csv')

# Tokenization
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
max_length = 128  # Adjust as needed

def tokenize_text(text):
    inputs = tokenizer(
        text,
        add_special_tokens=True,
        truncation=True,
        max_length=max_length,
        padding='max_length',
        return_tensors='tf'
    )
    return inputs

X = df['comment'].tolist()
y = df['ADR_Status'].tolist()

# Tokenize the text data
X_encoded = [tokenize_text(text) for text in X]

# Pad the sequences to have consistent lengths
X_input_ids = tf.keras.preprocessing.sequence.pad_sequences(
    [input_dict['input_ids'][0].numpy() for input_dict in X_encoded],
    maxlen=max_length,
    padding='post'
)

y = tf.convert_to_tensor(y, dtype=tf.int32)

# Split the data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(
    X_input_ids, y.numpy(), test_size=0.2, random_state=42
)

# Define your RoBERTa model
config = RobertaConfig.from_pretrained('roberta-base', num_labels=len(df['ADR_Status'].unique()))  # Adjust the number of classes accordingly
model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', config=config)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Train your model
epochs = 5  # Adjust as needed
batch_size = 10  # Adjust as needed

history = model.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size=batch_size,
    #validation_data=(X_val, y_val)
    validation_split=0.2
)

# Save the trained model
model.save_pretrained('fine_tuned_roberta_model')



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

Epoch 1/5
50/50 [==============================] - 950s 18s/step - loss: 0.2871 - accuracy: 0.9124 - val_loss: 0.1547 - val_accuracy: 0.9593
Epoch 2/5
50/50 [==============================] - 921s 18s/step - loss: 0.1899 - accuracy: 0.9124 - val_loss: 0.1937 - val_accuracy: 0.9350
Epoch 3/5
50/50 [==============================] - 911s 18s/step - loss: 0.1878 - accuracy: 0.9206 - val_loss: 0.1757 - val_accuracy: 0.9431
Epoch 4/5
50/50 [==============================] - 905s 18s/step - loss: 0.1766 - accuracy: 0.9185 - val_loss: 0.1657 - val_accuracy: 0.9512
Epoch 5/5
50/50 [==============================] - 884s 18s/step - loss: 0.1681 - accuracy: 0.9308 - val_loss: 0.1384 - val_accuracy: 0.9593


In [ ]:
# Assuming you have a test dataset (X_test, y_test) prepared similarly to the training data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)

print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


5/5 - 79s - loss: 0.1477 - accuracy: 0.9545 - 79s/epoch - 16s/step
Test Loss: 0.1476648449897766
Test Accuracy: 0.9545454382896423


In [ ]:
# from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
# precision = precision_score(test_true_labels, test_predictions, average='weighted')
# recall = recall_score(test_true_labels, test_predictions, average='weighted')
# f1 = f1_score(test_true_labels, test_predictions, average='weighted')

# print(f"Test Precision: {precision}")
# print(f"Test Recall: {recall}")
# print(f"Test F1-Score: {f1}")

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred.logits, axis=1)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred_classes)
recall = recall_score(y_test, y_pred_classes)
f1 = f1_score(y_test, y_pred_classes)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


5/5 [==============================] - 79s 15s/step
Precision: 0.8333333333333334
Recall: 0.45454545454545453
F1 Score: 0.5882352941176471


In [ ]:
# from sklearn.metrics import precision_score, recall_score, f1_score

# # Predict on the test set
# y_pred = model.predict(X_test)

# # Get the predicted class labels
# y_pred_labels = np.argmax(y_pred.logits, axis=1)

# # Calculate precision, recall, and F1-score
# precision = precision_score(y_test, y_pred_labels, average=None)
# recall = recall_score(y_test, y_pred_labels, average=None)
# f1 = f1_score(y_test, y_pred_labels, average=None)

# # Print the results for each class
# for class_label, prec, rec, f1_score in zip(range(len(precision)), precision, recall, f1):
#     print(f"Class {class_label} - Precision: {prec:.4f}, Recall: {rec:.4f}, F1-Score: {f1_score:.4f}")


In [ ]:
 from sklearn.metrics import classification_report, confusion_matrix
# y_pred = model.predict(X_test)

# # Get the predicted class labels
# y_pred_labels = np.argmax(y_pred.logits, axis=1)

# # Calculate precision, recall, F1-score, and support
# report = classification_report(y_test, y_pred_labels, target_names=['Class 0', 'Class 1'])
# print("Classification Report:\n", report)

# # Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)
 print("Confusion Matrix:\n", conf_matrix)

Confusion Matrix:
 [[142   1]
 [  6   5]]
